<a href="https://colab.research.google.com/github/hillelda/ANLP/blob/main/rec_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title pip install
! pip install datasets
! pip install evaluate
! pip install accelerate -U
! pip install transformers[torch]
! pip install torch
! pip install peft
! pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

In [2]:
# @title Imports
import evaluate
import numpy as np
from datasets import load_dataset
import transformers
from transformers import (AutoModelForSequenceClassification, AutoTokenizer)
import torch
from tqdm import tqdm

In [33]:
# @title Globals
MODE = 'regular' #@param ["regular", "lora"]

In [34]:
# @title load model
if MODE == 'regular':
  tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
  model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base').cuda()
elif MODE == 'lora':
  tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
model2 = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base').cuda()


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized

In [35]:
# @title data
def preprocess_function(examples):
    result = tokenizer(examples['sentence1'], examples['sentence2'], max_length=256, truncation=True, padding='max_length')
    return result

raw_datasets = load_dataset("nyu-mll/glue", 'mrpc')
raw_datasets = raw_datasets.map(preprocess_function,batched=True)

train_dataset = raw_datasets["train"]
eval_dataset = raw_datasets["validation"]

# train_dataset = train_dataset.select(range(300)) #training on 5k samples

# Set format for PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
eval_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

In [36]:
# @title Metric
metric = evaluate.load("accuracy",)

# def compute_metrics(p):
#     preds = np.argmax(p.predictions, axis=1)
#     return metric.compute(predictions=preds, references=p.label_ids)

def compute_metrics(preds, labels):
    pred_labels = np.argmax(preds, axis=1)
    return metric.compute(predictions=pred_labels, references=labels)

In [37]:
# @title Imports for Trainer alternative
from torch.utils.data import DataLoader
from torch.optim import Adam
from transformers import DataCollatorWithPadding
from transformers import get_scheduler


In [38]:
# from transformers import get_scheduler


# # @title Init trainer
# # training_args = TrainingArguments(output_dir='/tmp/', do_eval=True, do_train=True, num_train_epochs=3, per_device_train_batch_size=8, learning_rate =5e-5)
# # trainer = Trainer(
# #     model=model,
# #     args=training_args,
# #     train_dataset=train_dataset,
# #     eval_dataset=eval_dataset,
# #     compute_metrics=compute_metrics,
# #     tokenizer=tokenizer,
# # )


# def train(model, train_dataset, eval_dataset, tokenizer, num_epochs=5, learning_rate=5e-5, batch_size=16):
#     model.train()
#     model.cuda()
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     optim = Adam(model.parameters(), lr=learning_rate)
#     scheduler = get_scheduler(
#         "linear",
#         optim,
#         # num_warmup_steps=0,
#         num_training_steps=num_epochs * len(train_loader),
#         num_warmup_steps = int(0.1 * num_epochs * len(train_loader))
#     )
#     for epoch in range(num_epochs):
#         torch.cuda.empty_cache()
#         for batch in tqdm(train_loader):
#             optim.zero_grad()
#             input_ids = batch['input_ids'].cuda()
#             attention_mask = batch['attention_mask'].cuda()
#             labels = batch['label'].cuda()
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             loss = outputs.loss
#             loss.backward()
#             optim.step()
#             scheduler.step()
#         print("Epoch: " + str(epoch) + " - Loss: " + str(loss.item()))
#         model.eval()
#         eval_loader = DataLoader(eval_dataset, batch_size=batch_size)
#         all_preds = []
#         all_labels = []
#         for batch in tqdm(eval_loader):
#             input_ids = batch['input_ids'].cuda()
#             attention_mask = batch['attention_mask'].cuda()
#             labels = batch['label'].cuda()
#             with torch.no_grad():
#                 outputs = model(input_ids, attention_mask=attention_mask)
#             preds = torch.argmax(outputs.logits, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         accuracy = (all_preds == all_labels).mean()
#         print("Epochs: " + str(epoch + 1) + " - Learning Rate: " + str(learning_rate) + " - Batch Size: " + str(batch_size) + " - Accuracy: " + str(accuracy))
#     return model, {'accuracy': accuracy}

In [39]:
def train(model, train_dataset, eval_dataset, tokenizer, num_epochs=3, learning_rate=1e-4, batch_size=8):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            torch.cuda.empty_cache()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

            loss = outputs.loss
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval()
        eval_accuracy = 0
        eval_loss = 0
        with torch.no_grad():
            for batch in eval_loader:
                torch.cuda.empty_cache()
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['label'].to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                eval_loss += loss.item()
                pred = torch.argmax(outputs.logits, dim=1)
                eval_accuracy += (pred == labels).sum().item()

        eval_accuracy = eval_accuracy / len(eval_dataset)
        train_loss = train_loss / len(train_loader)
        eval_loss = eval_loss / len(eval_dataset)

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_loss:.4f}, Eval Accuracy: {eval_accuracy:.4f}, Eval Loss: {eval_loss:.4f}, LR: {learning_rate}, Batch Size: {batch_size}")

    final_accuracy = eval_accuracy
    return model, {'accuracy': final_accuracy}

In [ ]:
# @title Train!
# Train the model
if MODE == 'regular':
  trained_model, metrics = train(model, train_dataset, eval_dataset, tokenizer)
  metrics

Epoch 1/3: 100%|██████████| 459/459 [03:50<00:00,  1.99it/s]


Epoch 1/3 - Train Loss: 0.6379, Eval Accuracy: 0.6838, Eval Loss: 0.0781, LR: 0.0001, Batch Size: 8


Epoch 2/3: 100%|██████████| 459/459 [03:52<00:00,  1.97it/s]


Epoch 2/3 - Train Loss: 0.6332, Eval Accuracy: 0.6838, Eval Loss: 0.0781, LR: 0.0001, Batch Size: 8


Epoch 3/3:  18%|█▊        | 82/459 [00:42<03:11,  1.97it/s]

In [15]:
torch.cuda.empty_cache()

In [16]:
#@title Lora imports
import peft
from peft import LoraModel, LoraConfig

In [18]:
# print(model)

In [19]:
# @title Lora model base
#tokenizer2 = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')
#model2 = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base').cuda()

In [30]:
# @title Lora model config

lora_config = LoraConfig(
    r=32,
    lora_alpha=64, #should be about r*2.
    lora_dropout=0.05,
    task_type="classification",
    target_modules=['query_proj', 'value_proj'],
    modules_to_save=['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
)
lora_model = LoraModel(model2, lora_config, adapter_name="default")

In [31]:
# print(lora_model)

In [32]:
# @title Train LORA
if MODE == 'lora':
  trained_model, metrics = train(lora_model, train_dataset, eval_dataset, tokenizer, learning_rate=1e-4)
  metrics

Epoch 1/3: 100%|██████████| 459/459 [03:23<00:00,  2.26it/s]


Epoch 1/3 - Train Loss: 0.5674, Eval Accuracy: 0.7574, Eval Loss: 0.0578, LR: 0.0001, Batch Size: 8


Epoch 2/3: 100%|██████████| 459/459 [03:21<00:00,  2.27it/s]


Epoch 2/3 - Train Loss: 0.4403, Eval Accuracy: 0.8358, Eval Loss: 0.0567, LR: 0.0001, Batch Size: 8


Epoch 3/3: 100%|██████████| 459/459 [03:21<00:00,  2.27it/s]


Epoch 3/3 - Train Loss: 0.3472, Eval Accuracy: 0.8603, Eval Loss: 0.0415, LR: 0.0001, Batch Size: 8


In [ ]:
# @title Evaluate
# metrics = trainer.evaluate(eval_dataset=eval_dataset)
# metrics